Banco de Dados
==

Um banco de dados relacional é uma coleção de dados organizados em tabelas que permite inserção, modificação, remoção e consulta de dados de forma eficiente e segura.

Existem vários bancos de dados que podem ser usados em Python.  O que usaremos aqui é um banco, o SQLite3, existente na biblioteca padrão que é de fácil manutenção.

Banco de dados relacionais só trabalham com tabelas, que consistem de linhas e colunas.  As colunas contém os dados individuais, e devem ter nome, tipo e, opcionalmente, restrição.

In [ ]:
import sqlite3

#Esta é a forma de abrirmos um arquivo de sqlite
conn = sqlite3.connect('python2018.db')

# Bancos relacionais só trabalham com tabelas
try:
    conn.execute("CREATE TABLE alunos (CPF VARCHAR PRIMARY KEY, nome VARCHAR, "
        "sobrenome VARCHAR, nota REAL)")
except sqlite3.OperationalError:
    print("Tabela 'alunos' já existe. Ignorando erro...")

# Sempre que modificarmos a tabela temos que dar um commit() para salvar as modificações
conn.commit()
# O método close() desativa a conexão.  Nada mais pode ser feito no banco de dados após o close()
conn.close()

As operações efetuadas em banco de dados em geral é usando a linguagem SQL.  Um resumo desta linguagem pode ser encontrado [aqui](http://files.zeroturnaround.com/pdf/zt_sql_cheat_sheet.pdf).  Outros dois mais completos podem ser achados [aqui](http://www.sqltutorial.org/wp-content/uploads/2016/04/SQL-cheat-sheet.pdf) e [aqui](/www.sql-tutorial.net/sql-cheat-sheet.pdf).

Os comandos mais usados são os seguintes:
* INSERT - para inserir novos dados em uma tabela
* SELECT - para pesquisar dados em uma tabela
* UPDATE - para atualizar dados em uma tabela
* DELETE - para remover dados de uma tabela
* CREATE - para criar tabelas e views

In [ ]:
conn = sqlite3.connect('python2018.db')

# Inserindo um valor
try:
    conn.execute("INSERT INTO alunos VALUES ('000.000.000-00', 'Astrogildo', 'Silva', 8.0)")

# Inserindo um valor através de variáveis
    cpf, nome, sobrenome, nota = "111.111.111-11", "Godofredo", "Ramos", 9.0
    conn.execute("INSERT INTO alunos VALUES (?, ?, ?, ?)", (cpf, nome, sobrenome, nota))
except sqlite3.IntegrityError:
    print("Dado já existente...  Desfazendo modificações!!!")
    conn.rollback
else:
    print("Tudo ok.  Commitando...")
    conn.commit()

Além do tipo `Connection` retornado pela função `connect()`, o banco de dados também tem o tipo `Cursor`, que é o que é geralmente usado para executar as operações.

In [ ]:
c = conn.cursor()

c.execute("UPDATE alunos SET nota = 7.0 WHERE nome = 'Godofredo'")
c.execute("UPDATE alunos SET nota = 10.0 WHERE nome = 'Ermengarda'")
conn.commit()

In [ ]:
rows = c.execute("SELECT * FROM alunos")
for cpf, nome, sobrenome, nota in rows:
    print("CPF = {} - Nome = {} {}\t\tNota = {}".format(cpf, nome, sobrenome, nota))

In [ ]:
# Podemos inserir vários elementos de uma única vez usando o método executemany()
rows = [("222.222.222-22", "Rei", "Leão", 3.0),
       ("333.333.333-33", "Rainha", "Leoa", 9.0),
       ("444.444.444-44", "Ermengarda", "Assunção", 10.0)]
try:
    c.executemany("INSERT INTO alunos VALUES (?, ?, ?, ?)", rows)
except sqlite3.IntegrityError:
    print("Valores repetidos.  Desfazendo tudo...")
    conn.rollback()
else:
    print("Sucesso")
    conn.commit()

In [ ]:
rows = c.execute("SELECT * FROM alunos WHERE nota >= 9.0")
print("Aprovados:")
for cpf, nome, sobrenome, nota in rows:
    print("CPF = {} - Nome = {} {}\t\tNota = {}".format(cpf, nome, sobrenome, nota))
    print()